In [3]:
from config import Config
import networkx as nx
import json

papers = None
with open(Config.REDUCED_JSON_PATH, 'r') as infile:
    papers = json.load(infile)

unarxiv = None
with open(Config.UNARXIV_REDUCED_JSON_PATH, 'r') as infile:
    unarxiv = json.load(infile)

In [4]:
# Cursory density estimation

l = set(papers.keys())
found = {True: 0, False: 0}
citing = set()
cited = set()
all_citations = set()
for id, md in papers.items():
    for ref in md['arxiv_bib_ids']:
        all_citations.add(ref)
        if ref in l:
            found[True] += 1
            citing.add(id)
            cited.add(ref)
        else:
            found[False] += 1

print(f'''This dataset contains {len(papers)} papers making {sum(found.values())} citations \
of {len(all_citations)} unique works, {found[True]} of which reference other papers in the dataset.
{len(cited.intersection(citing))} papers in the dataset have both an incoming and outgoing citation in the set.''')

This dataset contains 795 papers making 10918 citations of 8947 unique works, 245 of which reference other papers in the dataset.
31 papers in the dataset have both an incoming and outgoing citation in the set.


In [5]:
# Ensure papers is a subset of unarxiv so that unarxiv's keys can be used alone for node generation
all([k in unarxiv for k in papers])

True

In [15]:
g = nx.DiGraph()
for id, data in unarxiv.items():
    for bib_id in data['arxiv_bib_ids']:
        if bib_id in unarxiv:
            g.add_edge(bib_id, id)

In [17]:
nx.find_cycle(g)

[('0806.2274', '0905.3378'), ('0905.3378', '0806.2274')]